In [1]:
# get spark session, 2g mem per executor
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
import os

# set python env
os.environ['PYSPARK_PYTHON'] = "/opt/conda3/envs/lab2/bin/python"
spark = SparkSession.builder \
    .appName("ProcessCattleProdText") \
    .master("spark://node01:10077") \
    .enableHiveSupport()\
    .config("spark.driver.memory", "2g") \
    .config("spark.executor.memory", "2g") \
    .config("spark.cores.max", "3") \
    .config("spark.sql.shuffle.partitions", "12") \
    .config("spark.sql.autoBroadcastJoinThreshold", "-1") \
    .getOrCreate()

sc = spark.sparkContext

23/05/31 13:34:09 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/05/31 13:34:11 WARN spark.SparkContext: Please ensure that the number of slots available on your executors is limited by the number of cores to task cpus and not another custom resource. If cores is not the limiting resource then dynamic allocation will not work properly!


In [2]:
# import jieba and set stop word set
import jieba
import jieba.analyse

stop_words_rdd = sc.textFile("hdfs:///user/spark_temp/stopwords.dat")
stop_words_set = set(stop_words_rdd.collect())

In [3]:
# define map functions 
from datetime import datetime

date_string = datetime.today().strftime('%Y-%m-%d')
# cut item name and description
def cut_name_and_desc(item):
    id, name, desc = item
    name_cut = set(jieba.cut(name))
    name_cut_pure = set(jieba.cut(name)) - stop_words_set
    desc_cut = set(jieba.cut(desc))
    desc_cut_pure = set(jieba.cut(desc)) - stop_words_set
    return (id,name_cut,name_cut_pure,desc_cut,desc_cut_pure)

# map item's cut list to word count
def to_count(item):
    id,name_cut,name_cut_pure,desc_cut,desc_cut_pure = item
    for i in name_cut_pure:
        yield ((id, i),1)
    for i in desc_cut_pure:
        yield ((id,i),1)
        
# trnasfer (id, key), count to id,key,count, date
def split_key_set_date(item):
    key1,count = item
    id,key = key1
    date = date_string
    return id,key,count,date

In [4]:
all_items = spark.sql("select id,name,description from item_ods where category='cattle_product'").rdd
print(all_items.count())

23/05/31 13:34:29 WARN session.SessionState: METASTORE_FILTER_HOOK will be ignored, since hive.security.authorization.manager is set to instance of HiveAuthorizerFactory.


112


In [5]:
# cut item's name and description
cut_items = all_items.map(cut_name_and_desc)
print(all_items.count())

112


In [6]:
# do word count
item_word_count = cut_items.flatMap(to_count)\
                    .reduceByKey(lambda a,b:a+b)\
                    .map(split_key_set_date)
print(item_word_count.count())

1422


In [7]:
# create a table for wordcount
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
schema = StructType([
    StructField("item_id", IntegerType(), True),
    StructField("key_word", StringType(), True),
    StructField("word_count", IntegerType(), True),
    StructField("date", StringType(),True)
])
item_word_count = spark.createDataFrame(item_word_count, schema)
item_word_count.createOrReplaceTempView("cattle_prod_word_count")
item_word_count.show()

+-------+--------+----------+----------+
|item_id|key_word|word_count|      date|
+-------+--------+----------+----------+
|     16|    优质|         2|2023-05-31|
|     16|    牛肉|         1|2023-05-31|
|     16|    健康|         1|2023-05-31|
|     16|    放养|         1|2023-05-31|
|     16|    口感|         1|2023-05-31|
|     16|  无污染|         1|2023-05-31|
|     16|    方式|         1|2023-05-31|
|     18|    牛奶|         1|2023-05-31|
|     18|    养生|         1|2023-05-31|
|     18|    特色|         1|2023-05-31|
|     18|    四平|         1|2023-05-31|
|     18|    优质|         1|2023-05-31|
|     18|    饮用|         1|2023-05-31|
|     18|    健康|         1|2023-05-31|
|     18|    营养|         1|2023-05-31|
|     18|    家庭|         1|2023-05-31|
|     18|  抗生素|         1|2023-05-31|
|     27|  牛肉干|         2|2023-05-31|
|     27|     123|         1|2023-05-31|
|     27|    鲜美|         1|2023-05-31|
+-------+--------+----------+----------+
only showing top 20 rows



In [8]:
# 将DataFrame写入Hive
item_word_count.write.mode("overwrite").partitionBy("date").saveAsTable("cattle_prod_word_count")
spark.sql("show tables").show()

+--------+--------------------+-----------+
|database|           tableName|isTemporary|
+--------+--------------------+-----------+
| default|cattle_prod_word_...|      false|
| default|           event_ods|      false|
| default|     item_fresh_list|      false|
| default|       item_hot_list|      false|
| default|            item_ods|      false|
| default|          item_order|      false|
| default|          item_score|      false|
| default|        item_tag_ods|      false|
| default|     item_word_count|      false|
| default|       item_word_idf|      false|
| default|        item_word_tf|      false|
| default|     item_word_tfidf|      false|
| default|             tag_ods|      false|
| default|                test|      false|
| default|               test2|      false|
| default|user_item_action_...|      false|
| default|            user_ods|      false|
| default|        user_tag_ods|      false|
| default|vet_twitte_conten...|      false|
| default|vet_twitte_title_...| 

In [9]:
spark.stop()